In [1]:
import os
import pandas as pd
import numpy as np
import json

In [2]:
data_folder = "../raw_data"

yellow_data_path = os.path.join(data_folder, "yellow_data.csv")
green_data_path = os.path.join(data_folder, "green_data.csv")
preprocessed_data_path = os.path.join(data_folder, "preprocessed_taxi_data.csv")
weather_data_path = os.path.join(data_folder, "weather_data.json")
merged_data_path = os.path.join(data_folder, "merged_data.csv")

CHUNK_SIZE = 10000

In [3]:
def preprocess_chunk(chunk, taxi_type, is_green=False):
    chunk['taxi_type'] = taxi_type
    # Standardize column names
    chunk.rename(columns={
        'tpep_pickup_datetime': 'pickup_datetime',
        'tpep_dropoff_datetime': 'dropoff_datetime',
        'lpep_pickup_datetime': 'pickup_datetime',
        'lpep_dropoff_datetime': 'dropoff_datetime',
    }, inplace=True)
    # Drop unnecessary columns for green taxis
    if is_green:
        chunk.drop(columns=['trip_type'], inplace=True)
        chunk.drop(columns=['ehail_fee'], inplace=True)
    chunk.dropna(inplace=True)
    return chunk

In [ ]:
is_first_chunk = True

# Process Green Taxi Data in Chunks
print("Processing Green Taxi Data...")
for green_chunk in pd.read_csv(green_data_path, chunksize=CHUNK_SIZE, low_memory=False):
    green_chunk = preprocess_chunk(green_chunk, taxi_type='green', is_green=True)
    green_chunk.to_csv(preprocessed_data_path, mode='w' if is_first_chunk else 'a', header=is_first_chunk, index=False)
    is_first_chunk = False

# Process Yellow Taxi Data in Chunks
print("Processing Yellow Taxi Data...")
for yellow_chunk in pd.read_csv(yellow_data_path, chunksize=CHUNK_SIZE, low_memory=False):
    yellow_chunk = preprocess_chunk(yellow_chunk, taxi_type='yellow')
    yellow_chunk.to_csv(preprocessed_data_path, mode='w' if is_first_chunk else 'a', header=is_first_chunk, index=False)
    is_first_chunk = False

print(f"Final preprocessed data saved to '{preprocessed_data_path}'")


In [8]:
with open(weather_data_path, 'r') as f:
    weather_json = json.load(f)
weather_data = weather_json["data"]
weather_df = pd.DataFrame(weather_data)
weather_df = weather_df.drop(columns=['tsun', 'wpgt'])


# Ensure weather data date format is correct and convert to datetime object
weather_df['date'] = pd.to_datetime(weather_df['date'], format='%Y-%m-%d', errors='coerce')

# Preview date formats
print(weather_df)


          date  tavg  tmin  tmax  prcp  snow   wdir  wspd    pres
0   2021-01-01   3.6   2.2   4.4  17.3   0.0   36.0  13.2  1029.4
1   2021-01-02   6.3   3.3  12.2   6.1   0.0  319.0  11.6  1012.9
2   2021-01-03   3.4   2.8   4.4   5.7   0.0   33.0  22.4  1017.1
3   2021-01-04   4.6   2.8   7.2   0.7   0.0    7.0   8.1  1015.0
4   2021-01-05   4.4   3.3   5.6   0.0   0.0    0.0   8.1  1013.5
..         ...   ...   ...   ...   ...   ...    ...   ...     ...
360 2021-12-27   1.4  -0.6   4.1   5.3   0.0   33.0   8.9  1017.2
361 2021-12-28   6.1   3.7   9.0   3.4   0.0  297.0   8.0  1010.3
362 2021-12-29   6.3   4.4   9.9   7.7   0.0   49.0   9.4  1012.0
363 2021-12-30   6.4   4.1   9.9   1.7   0.0   33.0   5.7  1013.9
364 2021-12-31   8.2   5.7  10.1   0.3   0.0  218.0   8.0  1013.9

[365 rows x 9 columns]


In [9]:
# Merge in chunks
chunk_size = 10000
is_first_chunk = True

print("Merging taxi data with weather data...")

for chunk in pd.read_csv(
    preprocessed_data_path,
    chunksize=chunk_size,
    parse_dates=['dropoff_datetime']):
    # Extract the date from dropoff_datetime and convert to datetime
    chunk['date'] = chunk['dropoff_datetime'].dt.normalize()  # Extract the date part

    # Ensure chunk['date'] is properly converted to datetime
    chunk['date'] = pd.to_datetime(chunk['date'], errors='coerce')

    # Ensure weather_df['date'] is also datetime before merging
    weather_df['date'] = pd.to_datetime(weather_df['date'], errors='coerce').dt.normalize()

    # Debugging: print first few values and types to check conversion
    print(f"chunk['date'] preview: {chunk['date'].head()}")

    # Merge the chunk with weather data on the 'date' column
    merged_chunk = chunk.merge(weather_df, how='left', on='date')

    # Save merged data incrementally
    merged_chunk.to_csv(merged_data_path, mode='w' if is_first_chunk else 'a', header=is_first_chunk, index=False)
    is_first_chunk = False

print(f"Merged data saved to '{merged_data_path}'")

Merging taxi data with weather data...


/tmp/ipykernel_66408/1221352572.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in pd.read_csv(


chunk['date'] preview: 0   2021-01-01
1   2021-01-01
2   2021-01-01
3   2021-01-01
4   2021-01-01
Name: date, dtype: datetime64[ns]


/tmp/ipykernel_66408/1221352572.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in pd.read_csv(


chunk['date'] preview: 10000   2021-01-08
10001   2021-01-08
10002   2021-01-09
10003   2021-01-09
10004   2021-01-08
Name: date, dtype: datetime64[ns]
chunk['date'] preview: 20000   2021-01-16
20001   2021-01-16
20002   2021-01-16
20003   2021-01-16
20004   2021-01-16
Name: date, dtype: datetime64[ns]


...

/tmp/ipykernel_66408/1221352572.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in pd.read_csv(


chunk['date'] preview: 30060000   2021-12-31
30060001   2021-12-31
30060002   2021-12-31
30060003   2021-12-31
30060004   2021-12-31
Name: date, dtype: datetime64[ns]


/tmp/ipykernel_66408/1221352572.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in pd.read_csv(


chunk['date'] preview: 30070000   2021-12-31
30070001   2021-12-31
30070002   2021-12-31
30070003   2021-12-31
30070004   2021-12-31
Name: date, dtype: datetime64[ns]
chunk['date'] preview: 30080000   2021-12-31
30080001   2021-12-31
30080002   2022-01-01
30080003   2021-12-31
30080004   2021-12-31
Name: date, dtype: datetime64[ns]
Merged data saved to '../raw_data/merged_data.csv'


/tmp/ipykernel_66408/1221352572.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in pd.read_csv(
